# Robotics Applications of ODE Solving with Julia

Welcome to this computational mathematics practical class notebook! Today, we will explore how Julia can be used to solve ordinary differential equations in the context of robotics. Our example model—a simple *double integrator*—can represent the dynamics of a robot moving along a straight line. We will use DifferentialEquations.jl along with several ODE solvers and compare their performance and behavior.

**Learning Objectives:**
- Learn how to set up a robotics simulation using ODE models in Julia.
- Experiment with different ODE solvers (e.g., Tsit5, Rodas5, CVODE_BDF) and see how they differ in handling non‑stiff and stiff systems.
- Understand the impact of solver choice on accuracy, stability, and execution time.

This notebook is designed to run on Binder
[Binder Link](https://mybinder.org/).

Let's get started!

In [ ]:
##############################
# Package Setup
##############################

using Pkg
# If running for the first time on Binder, uncomment the following lines:
# Pkg.add("DifferentialEquations")
# Pkg.add("Plots")

using DifferentialEquations
using Plots

# Set a default plot theme
default(; lw=2, size=(600,400))


## 1. Introducing the Robotics Model

In robotics, a common simple model is the **double integrator** which can represent a robot’s linear motion. Here, the state vector is defined as:

- **u[1]:** Position
- **u[2]:** Velocity

The dynamics are given by the equations:

- $$ \frac{d}{dt} (\text{position}) = \text{velocity} $$
- $$ \frac{d}{dt} (\text{velocity}) = -k \times (\text{position}) - c \times (\text{velocity}) $$

where $k$ is the spring coefficient and $c$ is the damping coefficient. This simple model will allow us to explore both non‑stiff and stiff behavior by adjusting the parameters.

In [ ]:
##############################
# Define the ODE Model
##############################

# The robot dynamics function for the double integrator
function robot_dynamics!(du, u, p, t)
    # u[1] = position, u[2] = velocity
    du[1] = u[2]                           # d(position)/dt = velocity
    du[2] = -p[1]*u[1] - p[2]*u[2]           # acceleration = -k*position - c*velocity
end

# Initial conditions: robot starts at position 1.0 with zero initial velocity
u0 = [1.0, 0.0]

# Time span for the simulation: 0 to 10 seconds
tspan = (0.0, 10.0)

# Parameters: p = [spring constant, damping coefficient]
p = [2.0, 0.5]

# Create the ODE problem
prob = ODEProblem(robot_dynamics!, u0, tspan, p)


## 2. Solving the ODE with Different Solvers

DifferentialEquations.jl offers a wide variety of solvers. In this section, we will solve the **double integrator** problem with different solvers and compare the results. You can refer to the [DifferentialEquations.jl Solvers documentation](https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/) for more details.

We will use:

- **Tsit5**: A non‑stiff, explicit Runge‑Kutta method
- **Rodas5**: An implicit method suitable for potentially stiff systems
- **CVODE_BDF**: A solver from the Sundials suite (often used in larger stiff problems)

Feel free to adjust parameters (e.g., increase the damping constant) to see how the solvers respond to stiffer dynamics.

In [ ]:
##############################
# Solve using Tsit5 (non-stiff)
##############################

sol_Tsit5 = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

plot(sol_Tsit5.t, sol_Tsit5[1,:], label="Tsit5", xlabel="Time (s)", ylabel="Position", 
     title="Double Integrator Position vs Time", lw=2)
display(current())


In [ ]:
##############################
# Solve using Rodas5 (for stiff systems)
##############################

sol_Rodas5 = solve(prob, Rodas5(), reltol=1e-8, abstol=1e-8)

# Plotting on the same figure for comparison
plot(sol_Tsit5.t, sol_Tsit5[1,:], label="Tsit5", xlabel="Time (s)", ylabel="Position", 
     title="Double Integrator: Position Comparison", lw=2)
plot!(sol_Rodas5.t, sol_Rodas5[1,:], label="Rodas5", lw=2)
display(current())


In [ ]:
##############################
# Solve using CVODE_BDF (from Sundials, for stiff problems)
##############################

sol_CVODE = solve(prob, CVODE_BDF(), reltol=1e-8, abstol=1e-8)

# Add the solution from CVODE_BDF on our comparison plot
plot!(sol_CVODE.t, sol_CVODE[1,:], label="CVODE_BDF", linestyle=:dash, lw=2)
display(current())


## 3. Experimentation and Analysis

Now it’s your turn to experiment! Try the following tasks:

1. **Vary the Parameters:** Increase the damping coefficient (or spring constant) to push the model towards stiffness. Notice how the solutions change and how some solvers handle the stiff dynamics better than others.

2. **Solver Tolerances:** Modify the relative (`reltol`) and absolute (`abstol`) tolerances when calling `solve()`. Observe the changes in accuracy and computation time.

3. **Timing Analysis:** Use the `@time` macro to measure how long each solver takes. For example:

   ```julia
   @time solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)
   @time solve(prob, Rodas5(), reltol=1e-8, abstol=1e-8)
   ```

4. **Discussion:** Answer the following in your own words in a Markdown cell below (or as instructor-led discussion):
   - How does the choice of solver affect the solution for non-stiff vs. stiff regimes?
   - What trade-offs do you observe in terms of computation time, accuracy, and stability?

For more details on the solvers and their appropriate use cases, visit the [DifferentialEquations.jl Solvers page](https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/).

## 4. TORA Systems: 1-DoF and 2-DoF Examples

In this section, we explore the **TORA system**—a classic model in control and robotics that represents oscillatory behavior with rotational actuation. We consider two cases:

1. **1-DoF TORA:** A single degree‑of‑freedom torsional oscillator where the dynamics follow
   $$
   \theta'' + d\,\theta' + c\,\sin(\theta) = \text{input},
   $$
   with free oscillation (i.e. zero input) for demonstration.

2. **2-DoF TORA:** A coupled system of two oscillators. Here, the dynamics are modeled as
   $$
   \begin{aligned}
   \theta_1'' + d_1\,\theta_1' + c_1\,\sin(\theta_1) + k(\theta_1-\theta_2) &= \text{input}_1, \\
   \theta_2'' + d_2\,\theta_2' + c_2\,\sin(\theta_2) - k(\theta_1-\theta_2) &= \text{input}_2,
   \end{aligned}
   $$
   where the coupling term $ k (\theta_1 - \theta_2) $ links the two oscillators.

Both demonstrations use the DifferentialEquations.jl package to solve the ODEs and Plots.jl for visualization. Try modifying the parameters or switching to a different ODE solver to explore how stiffness or nonlinearity influences the dynamics!

In [ ]:
##############################
# TORA 1-DoF Model
##############################

# The ODE for a 1-DoF TORA system.
# Equation: θ'' + d θ' + c * sin(θ) = input, (we set input = 0 for free oscillation)
    function tora1dof!(du, u, p, t)
        d, c, input = p  # damping, stiffness, and external input torque
        theta, omega = u
        du[1] = omega
        du[2] = -d * omega - c * sin(theta) + input
    end
    
    # Initial condition: small initial displacement, starting from rest.
    u0_tora1 = [0.2, 0.0]
    # Time span: simulate over 30 seconds.
    tspan_tora1 = (0.0, 30.0)
    # Parameters: [damping, stiffness, input]. Set input = 0 for free oscillation.
    p_tora1 = [0.1, 3.0, 0.0]
    
    # Define and solve the ODE problem
    prob_tora1 = ODEProblem(tora1dof!, u0_tora1, tspan_tora1, p_tora1)
    sol_tora1 = solve(prob_tora1, Tsit5(), reltol=1e-8, abstol=1e-8)
    
    # Visualize the results:
    # (a) θ vs. Time
    plt1 = plot(sol_tora1.t, sol_tora1[1,:],
                label="θ (angle)",
                xlabel="Time (s)",
                ylabel="θ",
                title="TORA 1-DoF: Angular Displacement vs Time",
                lw=2)
    
    # (b) Phase Portrait: θ vs. ω
    plt2 = plot(sol_tora1[1,:], sol_tora1[2,:],
                label="Phase Portrait",
                xlabel="θ (angle)",
                ylabel="ω (angular velocity)",
                title="TORA 1-DoF: Phase Portrait",
                lw=2,
                legend=:bottomright)
    
    display(plt1)
    display(plt2)

##############################
# TORA 2-DoF Model
##############################

# The ODE for a 2-DoF coupled TORA system.
# Equations:
# θ₁'' + d₁ θ₁' + c₁ sin(θ₁) + k*(θ₁ - θ₂) = input₁,
# θ₂'' + d₂ θ₂' + c₂ sin(θ₂) - k*(θ₁ - θ₂) = input₂.
function tora2dof!(du, u, p, t)
    # Unpack parameters:
    d1, c1, d2, c2, k, input1, input2 = p
    # Unpack state variables:
    theta1, theta2, omega1, omega2 = u
    du[1] = omega1
    du[2] = omega2
    du[3] = -d1 * omega1 - c1 * sin(theta1) + k*(theta2 - theta1) + input1
    du[4] = -d2 * omega2 - c2 * sin(theta2) - k*(theta2 - theta1) + input2
end

# Initial condition: small displacements and zero initial velocities.
u0_tora2 = [0.2, 0.25, 0.0, 0.0]
# Time span: simulate over 40 seconds.
tspan_tora2 = (0.0, 40.0)
# Parameters: [d1, c1, d2, c2, coupling stiffness k, input1, input2] (inputs are 0 for free oscillation).
p_tora2 = [0.1, 3.0, 0.1, 3.0, 2.0, 0.0, 0.0]

# Define and solve the ODE problem
prob_tora2 = ODEProblem(tora2dof!, u0_tora2, tspan_tora2, p_tora2)
sol_tora2 = solve(prob_tora2, Tsit5(), reltol=1e-8, abstol=1e-8)

# Visualize the results:
# (a) Angular displacement vs. Time for both angles
plt3 = plot(sol_tora2.t, sol_tora2[1,:],
            label="θ₁",
            xlabel="Time (s)",
            ylabel="θ",
            title="TORA 2-DoF: Angular Displacement vs Time",
            lw=2)
plot!(sol_tora2.t, sol_tora2[2,:], label="θ₂", lw=2)

# (b) Phase portrait for the first oscillator (θ₁ vs. ω₁)
plt4 = plot(sol_tora2[1,:], sol_tora2[3,:],
            label="Phase Portrait θ₁ vs ω₁",
            xlabel="θ₁",
            ylabel="ω₁",
            title="TORA 2-DoF: Phase Portrait for Oscillator 1",
            lw=2,
            legend=:bottomright)

display(plt3)
display(plt4)

## 5. Further Exploration

Once you’re comfortable with this model, consider extending the notebook by:

- Modeling a multi‑link robotic arm, where the state vector includes joint angles and angular velocities.
- Incorporating feedback control (such as a PID controller) into the ODE to simulate closed‑loop dynamics.
- Exploring event handling (for example, when the robot reaches a target or collides with an object) using callbacks in DifferentialEquations.jl.

These exercises will deepen your understanding of how ODE solvers behave in a robotics context and how to choose an appropriate solver for your specific problem.

## 6. Conclusion

In this practical class, you learned how to set up a robotics model in Julia, solve it using multiple ODE solvers, and analyze the impact of solver choice on system behavior. Understanding these differences is important when scaling problems to more complex robotics applications.

Happy coding and exploring!